In [1]:
'''
created by Yufei Gao at 4/12/2018
'''

'\ncreated by Yufei Gao at 4/12/2018\n'

In [3]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import multiprocessing
import matplotlib.pyplot as plt

In [4]:
class data_proc():
    
    def process_zip():
        #dealing with zipcode
        zip_data=pd.read_csv('ZIP.csv')
        zip_data['Zip']=zip_data['Zip'].astype(str)
        for i in range(len(zip_data['Zip'])):
            if len(zip_data['Zip'][i])==4:
                zip_data['Zip'][i]='0'+zip_data['Zip'][i]
        zip_data['Zip']=[zip_data['Zip'][i][0:3] for i in range(len(zip_data))] 
        zip_data.iloc[:,-3:]=zip_data[['Median','Mean','Pop']].apply(lambda x: x.str.replace(',',''))
        for i in range(1,4):
            zip_data.iloc[:,-i]=pd.to_numeric(zip_data.iloc[:,-i],errors='coerce')
        zip_data['weight']=zip_data['Pop']/zip_data.groupby('Zip')['Pop'].transform(sum)
        zip_data['new_mean']=zip_data['Mean']*zip_data['weight']
        zip_data['new_median']=zip_data['Median']*zip_data['weight']
        zip_new=pd.DataFrame()
        zip_new=zip_data.groupby('Zip')['new_mean','new_median'].sum()
        return zip_new
        
    def readcsv():
        LARGE_FILE = "C:\\Users\Administrator\Desktop\practicum_regression\loan_data_no_current_converted.csv"
        CHUNKSIZE = 100000 # processing 100,000 rows at a time
        reader = pd.read_csv(LARGE_FILE, chunksize=CHUNKSIZE, low_memory=False)
        frames = []
        for df in reader:
            frames.append(df)
        loan_data = pd.concat(frames)
        return loan_data   
    def cleaning(df,zip_new,keep_desc=True,categorical_to_binary=True):
        df=df.dropna(axis=0,how='all')
        drop_list=['emp_title','title','earliest_cr_line','desc','issue_d','id','member_id','url','grade','sub_grade',
                   'int_rate','avg_cur_bal','addr_state','funded_amnt','funded_amnt_inv','collection_recovery_fee',
                   'collections_12_mths_ex_med','mths_since_last_major_derog','next_pymnt_d','recoveries','total_pymnt',
                   'total_pymnt_inv','total_rec_int','issue_d',' last_credit_pull_d','last_pymnt_d','last_credit_pull_d',
                  'total_rec_prncp','settlement_status','hardship_loan_status','hardship_status','debt_settlement_flag',
                   'verification_status','total_rec_late_fee']
        df.drop(drop_list,inplace=True,axis=1,errors='ignore')
        #deal with percentage mark
        df['revol_util']=df['revol_util'].replace('%','',regex=True).astype('float')/100
        #merge zipcode with census data
        df['zip_code']=df['zip_code'].apply(lambda x: x[:3])
        df=df.join(zip_new,on='zip_code')
        df.drop('zip_code',inplace=True,axis=1)
        #drop the observation that was missing for ALL field
        df=df.dropna(axis=0,how='all')
        #drop the features for which greater than 10% of the loans were missing data for
        num_rows=df.count(axis=0)
        df=df.iloc[:,(num_rows>=0.9*len(df)).tolist()]
        #drop the observation that was missing for any field
        df=df.dropna(axis=0,how='any')
        #label the dataset to create y
        #0:fully paid, does not meet policy:fully paid
        #1:Does not meet the credit policy. Status:Charged Off",default,charge of
        df=df[(True^df['loan_status'].isin([4]))] 
        df=df[(True^df['loan_status'].isin([5]))]
        df=df[(True^df['loan_status'].isin([6]))] 
        #label the dataset to create y
        y=df['loan_status'].replace(1,0)
        y=y.replace(2,0)
        y=y.replace(3,1)
        y=y.replace(7,1)
        y=y.replace(8,1)
        df=df.drop(['loan_status'],axis=1) 
        return df,y

In [5]:
zipdata = data_proc.process_zip()
df = data_proc.readcsv()
x,y = data_proc.cleaning(df,zipdata,keep_desc=False,categorical_to_binary=True)

In [6]:
len(x),len(y)

(772086, 772086)

In [31]:
feature=[]
for key, value in x.iteritems():
    feature.append(key)
    print (key)
len(feature)

loan_amnt
term
installment
emp_length
home_ownership
annual_inc
pymnt_plan
purpose
dti
delinq_2yrs
inq_last_6mths
open_acc
pub_rec
revol_bal
revol_util
total_acc
initial_list_status
out_prncp
out_prncp_inv
last_pymnt_amnt
policy_code
application_type
acc_now_delinq
tot_coll_amt
tot_cur_bal
total_rev_hi_lim
acc_open_past_24mths
bc_open_to_buy
bc_util
chargeoff_within_12_mths
delinq_amnt
mo_sin_old_rev_tl_op
mo_sin_rcnt_rev_tl_op
mo_sin_rcnt_tl
mort_acc
mths_since_recent_bc
num_accts_ever_120_pd
num_actv_bc_tl
num_actv_rev_tl
num_bc_sats
num_bc_tl
num_il_tl
num_op_rev_tl
num_rev_accts
num_rev_tl_bal_gt_0
num_sats
num_tl_30dpd
num_tl_90g_dpd_24m
num_tl_op_past_12m
pct_tl_nvr_dlq
percent_bc_gt_75
pub_rec_bankruptcies
tax_liens
tot_hi_cred_lim
total_bal_ex_mort
total_bc_limit
total_il_high_credit_limit
hardship_flag
hardship_type
hardship_reason
disbursement_method
new_mean
new_median


63

Binary Logistic Regression(Ridge) 

In [7]:
#%% Binary logistic regression
class log_reg():
    # Evaluate the model by splitting into train and test sets
    def split(x,y,rand=0):
        
        y = np.ravel(y)
        x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25,random_state=rand)
        
        return x_train, x_test, y_train, y_test 
    #we need to add validation dataset here
    
    # Find binary column method one
    def bool_cols(df,isbool=True):
        bool_cols=[]
        for col in df:
            if isbool==True:
                if df[col].dropna().value_counts().index.isin([0,1]).all():
                    bool_cols.append(col)
            else:
                if not df[col].dropna().value_counts().index.isin([0,1]).all():
                    bool_cols.append(col)
        return bool_cols
    # this above step is to facilitate normalization later
    # method two
    def not_bi(x):
        not_bi=[]
        for i in list(x):
            u=x[i].unique()
            if not (0 in u and 1 in u and len(u)==2): #if not binary
                not_bi.append(i)
        return not_bi
    
    def reg(x_train, y_train, c, num_iter):
           
        model = LogisticRegression(penalty='l2',class_weight='balanced',solver='sag',n_jobs=-1, C=c, max_iter = num_iter)
        
        """
        Why we need standardize?
        
        Note that ‘sag’ and ‘saga’ fast convergence is only guaranteed on features 
        with approximately the same scale. You can preprocess the data with 
        a scaler from sklearn.preprocessing.
        """
        
        model = model.fit(x_train, y_train)
        
        return model
    
    def ModelValuation(x_test,y_test,model):
        
        probs = model.predict_proba(x_test)
        #fpr, tpr, thresholds = metrics.roc_curve(y_test, probs[:, 1])
        
        #plt.figure(1)
        #plt.plot(fpr, tpr, label='LogisticRegression')
        #plt.xlabel('False positive rate')
        #plt.ylabel('True positive rate')
        #plt.title('ROC curve')
        #plt.legend(loc='best')
        #plt.show()
        
        #print("Area Under the Curve (AUC) from prediction score is %f" % metrics.roc_auc_score(y_test, probs[:, 1]))
    
        return None  
    
    def y_pred(x_test,threshold=0.5):
        
        if threshold == 0.5:
            y_predicted = model.predict(x_test)
        else:
            probs = model.predict_proba(x_test)
            y_predicted = np.array(probs[:,1] >= threshold).astype(int)
        
        return y_predicted    
    
    def GetScores(y_test,y_predicted):
        #G means score 
        CM = metrics.confusion_matrix(y_test, y_predicted)
        TN = CM[0,0]
        FN = CM[1,0]
        TP = CM[1,1]
        FP = CM[0,1]
        
        sensitivity = float(TP)/float(TP+FN)
        specificity = float(TN)/float(TN+FP)
        G = np.sqrt(sensitivity*specificity)
        #print("G score is %f" % G)
        #print("Specificity is %f" % specificity)
        
        # Generate and display different evaluation metrics
        print("Mean accuracy score is %f" % metrics.accuracy_score(y_test, y_predicted))
          
        #print("Confusion Marix")
        #print(CM)
        
        return specificity , G
        
    # Convenience function to plot confusion matrix
    def confusion(y_test,y_predicted,title):
        
        # Define names for the three Iris types
        names = ['Default', 'Not Default']
    
        # Make a 2D histogram from the test and result arrays
        pts, xe, ye = np.histogram2d(y_test, y_predicted, bins=2)
    
        # For simplicity we create a new DataFrame
        pd_pts = pd.DataFrame(pts.astype(int), index=names, columns=names )
        
        # Display heatmap and add decorations
        hm = sns.heatmap(pd_pts, annot=True, fmt="d")
        hm.axes.set_title(title)
        
        return None
            
    def find_threshold(x_test,y_test):
    
        probs = model.predict_proba(x_test)
        fpr, tpr, thresholds = metrics.roc_curve(y_test, probs[:, 1])
        
        sensitivity = tpr
        specificity = 1 - fpr
        G = np.sqrt(sensitivity*specificity)
        
        plt.figure(2)
        plt.plot(thresholds,G)
        plt.xlabel('Thresholds')
        plt.ylabel('G-Scores')
        plt.title('G-Scores with different thresholds')
        plt.show()
        
        
        print("The highest G score is %f with threshold at %f" % (np.amax(G),thresholds[np.argmax(G)]) )
        
        return thresholds[np.argmax(G)]
    # this is just testing, we add weight so we don't need to adjust threshold

Run Binary logistic Regression(Ridge)

Test

In [8]:
def validation_dataset(df, y, n, m):
    y=pd.Series(y)
    # by calling the function split, y will become array type
    interval_len=int(len(df)/n)
    if m == 1:
        x_test = df.iloc[0:interval_len]
        y_test = y.iloc[0:interval_len]
        x_train = df.iloc[interval_len:len(df)]
        y_train = y.iloc[interval_len:len(df)]
    elif m == n:
        x_test = df.iloc[(n-1)*interval_len:len(df)]
        y_test = y.iloc[(n-1)*interval_len:len(df)]
        x_train = df.iloc[0:(n-1)*interval_len]
        y_train = y.iloc[0:(n-1)*interval_len]
    else:
        x_test = df.iloc[(m-1)*interval_len:m*interval_len]
        x_train1 = df.iloc[0:(m-1)*interval_len]
        x_train2 = df.iloc[m*interval_len:len(df)]
        frames1 = [x_train1, x_train2]
        x_train = pd.concat(frames1)
        
        y_test = y.iloc[(m-1)*interval_len:m*interval_len]
        y_train1 = y.iloc[0:(m-1)*interval_len]
        y_train2 = y.iloc[m*interval_len:len(df)]
        frames2 = [y_train1, y_train2]
        y_train = pd.concat(frames2)
    return x_train, x_test, y_train, y_test

In [9]:
def drop_choice(df, y, n):
    if (len(df) % n != 0):
        df1 = df[0:len(df)-len(df)%n]
        y1 = y[0:len(y)-len(y)%n]
    return df1, y1

In [10]:
from sklearn.model_selection import train_test_split , cross_val_score
def split(x,y,rand=0):
        
        y = np.ravel(y)
        x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25,random_state=rand)
        
        return x_train, x_test, y_train, y_test

Validation

This part we focus on validation on a fixed test set

In [14]:
x,y=drop_choice(x, y, 10)

In [15]:
len(x),len(y)

(772080, 772080)

In [24]:

# Normalize
for i in range(1,6):
    not_bi = log_reg.not_bi(x)

    x_train_temp, x_test, y_train_temp, y_test = log_reg.split(x,y,rand=0)

    x_train, x_val_test, y_train, y_val_test = validation_dataset(x_train_temp,y_train_temp,5,i)

    scaler = StandardScaler()
    scaler.fit(x_train[not_bi]) 

    x_train_scaled=x_train
    x_test_scaled=x_test

    x_train_scaled[not_bi] = scaler.transform(x_train[not_bi])
    x_test_scaled[not_bi]  = scaler.transform(x_test[not_bi])

# Fit model
    model = log_reg.reg(x_train_scaled,y_train,1.0,100)
# Evaluate model
    log_reg.ModelValuation(x_test_scaled,y_test,model)
    y_predicted = log_reg.y_pred(x_test_scaled,threshold=0.5)
    spec , G = log_reg.GetScores(y_test,y_predicted)
#log_reg.confusion(y_test,y_predicted,'Default Confusion Matrix')

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
F:\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

Mean accuracy score is 0.833024
Mean accuracy score is 0.837454
Mean accuracy score is 0.832879
Mean accuracy score is 0.832605
Mean accuracy score is 0.832765


Chose a very small C=0.01

In [20]:
# Normalize
for i in range(1,11):
    not_bi = log_reg.not_bi(x)
    # x_test_real and y_test_real is the true test set
    # in the turning parameter process, we will not use the test set at this moment
    # notice that the rand state must be hold 
    x_train_temp, x_test_real, y_train_temp, y_test_real = log_reg.split(x,y,rand=0)
    # in this part, we cut the train into the real train and validation set
    x_train, x_test, y_train, y_test = validation_dataset(x_train_temp,y_train_temp,10,i)

    scaler = StandardScaler()
    scaler.fit(x_train[not_bi]) 

    x_train_scaled=x_train
    x_test_scaled=x_test

    x_train_scaled[not_bi] = scaler.transform(x_train[not_bi])
    x_test_scaled[not_bi]  = scaler.transform(x_test[not_bi])

# Fit model
    model = log_reg.reg(x_train_scaled,y_train,0.01,100)
# Evaluate model
    log_reg.ModelValuation(x_test_scaled,y_test,model)
    y_predicted = log_reg.y_pred(x_test_scaled,threshold=0.5)
    spec , G = log_reg.GetScores(y_test,y_predicted)
#log_reg.confusion(y_test,y_predicted,'Default Confusion Matrix')

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
F:\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

Mean accuracy score is 0.830501
Mean accuracy score is 0.832176
Mean accuracy score is 0.833178
Mean accuracy score is 0.833990
Mean accuracy score is 0.832073
Mean accuracy score is 0.834629
Mean accuracy score is 0.835233
Mean accuracy score is 0.830311
Mean accuracy score is 0.832556
Mean accuracy score is 0.831606


Choose a smaller C(C=0.1)

Notice that this part only shows you the mean accuracy metric, if you want other metric, just cancel the notation in the regresson class

Will take 20-30 min to run next cell

In [16]:
# Normalize
for i in range(1,11):
    not_bi = log_reg.not_bi(x)
    # x_test_real and y_test_real is the true test set
    # in the turning parameter process, we will not use the test set at this moment
    # notice that the rand state must be hold 
    x_train_temp, x_test_real, y_train_temp, y_test_real = log_reg.split(x,y,rand=0)
    # in this part, we cut the train into the real train and validation set
    x_train, x_test, y_train, y_test = validation_dataset(x_train_temp,y_train_temp,10,i)

    scaler = StandardScaler()
    scaler.fit(x_train[not_bi]) 

    x_train_scaled=x_train
    x_test_scaled=x_test

    x_train_scaled[not_bi] = scaler.transform(x_train[not_bi])
    x_test_scaled[not_bi]  = scaler.transform(x_test[not_bi])

# Fit model
    model = log_reg.reg(x_train_scaled,y_train,0.1,100)
# Evaluate model
    log_reg.ModelValuation(x_test_scaled,y_test,model)
    y_predicted = log_reg.y_pred(x_test_scaled,threshold=0.5)
    spec , G = log_reg.GetScores(y_test,y_predicted)
#log_reg.confusion(y_test,y_predicted,'Default Confusion Matrix')

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
F:\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

Mean accuracy score is 0.830415
Mean accuracy score is 0.832228
Mean accuracy score is 0.833264
Mean accuracy score is 0.834646
Mean accuracy score is 0.832090
Mean accuracy score is 0.834611
Mean accuracy score is 0.835406
Mean accuracy score is 0.830259
Mean accuracy score is 0.832936
Mean accuracy score is 0.831606


C=1

In [21]:
# Normalize
for i in range(1,11):
    not_bi = log_reg.not_bi(x)
    # x_test_real and y_test_real is the true test set
    # in the turning parameter process, we will not use the test set at this moment
    # notice that the rand state must be hold 
    x_train_temp, x_test_real, y_train_temp, y_test_real = log_reg.split(x,y,rand=0)
    # in this part, we cut the train into the real train and validation set
    x_train, x_test, y_train, y_test = validation_dataset(x_train_temp,y_train_temp,10,i)

    scaler = StandardScaler()
    scaler.fit(x_train[not_bi]) 

    x_train_scaled=x_train
    x_test_scaled=x_test

    x_train_scaled[not_bi] = scaler.transform(x_train[not_bi])
    x_test_scaled[not_bi]  = scaler.transform(x_test[not_bi])

# Fit model
    model = log_reg.reg(x_train_scaled,y_train,1,100)
# Evaluate model
    log_reg.ModelValuation(x_test_scaled,y_test,model)
    y_predicted = log_reg.y_pred(x_test_scaled,threshold=0.5)
    spec , G = log_reg.GetScores(y_test,y_predicted)
#log_reg.confusion(y_test,y_predicted,'Default Confusion Matrix')

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
F:\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

Mean accuracy score is 0.830259
Mean accuracy score is 0.832176
Mean accuracy score is 0.833247
Mean accuracy score is 0.834784
Mean accuracy score is 0.832124
Mean accuracy score is 0.834629
Mean accuracy score is 0.835354
Mean accuracy score is 0.830311
Mean accuracy score is 0.832867
Mean accuracy score is 0.831572


C=2

In [22]:
# Normalize
for i in range(1,11):
    not_bi = log_reg.not_bi(x)
    # x_test_real and y_test_real is the true test set
    # in the turning parameter process, we will not use the test set at this moment
    # notice that the rand state must be hold 
    x_train_temp, x_test_real, y_train_temp, y_test_real = log_reg.split(x,y,rand=0)
    # in this part, we cut the train into the real train and validation set
    x_train, x_test, y_train, y_test = validation_dataset(x_train_temp,y_train_temp,10,i)

    scaler = StandardScaler()
    scaler.fit(x_train[not_bi]) 

    x_train_scaled=x_train
    x_test_scaled=x_test

    x_train_scaled[not_bi] = scaler.transform(x_train[not_bi])
    x_test_scaled[not_bi]  = scaler.transform(x_test[not_bi])

# Fit model
    model = log_reg.reg(x_train_scaled,y_train,2,100)
# Evaluate model
    log_reg.ModelValuation(x_test_scaled,y_test,model)
    y_predicted = log_reg.y_pred(x_test_scaled,threshold=0.5)
    spec , G = log_reg.GetScores(y_test,y_predicted)
#log_reg.confusion(y_test,y_predicted,'Default Confusion Matrix')

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
F:\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

Mean accuracy score is 0.830277
Mean accuracy score is 0.832176
Mean accuracy score is 0.833247
Mean accuracy score is 0.834836
Mean accuracy score is 0.832107
Mean accuracy score is 0.834629
Mean accuracy score is 0.835354
Mean accuracy score is 0.830294
Mean accuracy score is 0.832867
Mean accuracy score is 0.831572


C=0.5

In [23]:
# Normalize
for i in range(1,11):
    not_bi = log_reg.not_bi(x)
    # x_test_real and y_test_real is the true test set
    # in the turning parameter process, we will not use the test set at this moment
    # notice that the rand state must be hold 
    x_train_temp, x_test_real, y_train_temp, y_test_real = log_reg.split(x,y,rand=0)
    # in this part, we cut the train into the real train and validation set
    x_train, x_test, y_train, y_test = validation_dataset(x_train_temp,y_train_temp,10,i)

    scaler = StandardScaler()
    scaler.fit(x_train[not_bi]) 

    x_train_scaled=x_train
    x_test_scaled=x_test

    x_train_scaled[not_bi] = scaler.transform(x_train[not_bi])
    x_test_scaled[not_bi]  = scaler.transform(x_test[not_bi])

# Fit model
    model = log_reg.reg(x_train_scaled,y_train,0.5,100)
# Evaluate model
    log_reg.ModelValuation(x_test_scaled,y_test,model)
    y_predicted = log_reg.y_pred(x_test_scaled,threshold=0.5)
    spec , G = log_reg.GetScores(y_test,y_predicted)
#log_reg.confusion(y_test,y_predicted,'Default Confusion Matrix')

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
F:\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

Mean accuracy score is 0.830328
Mean accuracy score is 0.832176
Mean accuracy score is 0.833247
Mean accuracy score is 0.834767
Mean accuracy score is 0.832107
Mean accuracy score is 0.834629
Mean accuracy score is 0.835354
Mean accuracy score is 0.830259
Mean accuracy score is 0.832884
Mean accuracy score is 0.831572


Choose a larger C (C=5)

In [17]:
# Normalize
for i in range(1,11):
    not_bi = log_reg.not_bi(x)

    x_train_temp, x_test_real, y_train_temp, y_test_real = log_reg.split(x,y,rand=0)

    x_train, x_test, y_train, y_test = validation_dataset(x_train_temp,y_train_temp,10,i)

    scaler = StandardScaler()
    scaler.fit(x_train[not_bi]) 

    x_train_scaled=x_train
    x_test_scaled=x_test

    x_train_scaled[not_bi] = scaler.transform(x_train[not_bi])
    x_test_scaled[not_bi]  = scaler.transform(x_test[not_bi])

# Fit model
    model = log_reg.reg(x_train_scaled,y_train,5,100)
# Evaluate model
    log_reg.ModelValuation(x_test_scaled,y_test,model)
    y_predicted = log_reg.y_pred(x_test_scaled,threshold=0.5)
    spec , G = log_reg.GetScores(y_test,y_predicted)
#log_reg.confusion(y_test,y_predicted,'Default Confusion Matrix')

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
F:\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

Mean accuracy score is 0.830294
Mean accuracy score is 0.832176
Mean accuracy score is 0.833247
Mean accuracy score is 0.834836
Mean accuracy score is 0.832107
Mean accuracy score is 0.834629
Mean accuracy score is 0.835354
Mean accuracy score is 0.830311
Mean accuracy score is 0.832867
Mean accuracy score is 0.831589


Chose C = 10

In [18]:
for i in range(1,11):
    not_bi = log_reg.not_bi(x)

    x_train_temp, x_test_real, y_train_temp, y_test_real = log_reg.split(x,y,rand=0)

    x_train, x_test, y_train, y_test = validation_dataset(x_train_temp,y_train_temp,10,i)

    scaler = StandardScaler()
    scaler.fit(x_train[not_bi]) 

    x_train_scaled=x_train
    x_test_scaled=x_test

    x_train_scaled[not_bi] = scaler.transform(x_train[not_bi])
    x_test_scaled[not_bi]  = scaler.transform(x_test[not_bi])

# Fit model
    model = log_reg.reg(x_train_scaled,y_train,10,100)
# Evaluate model
    log_reg.ModelValuation(x_test_scaled,y_test,model)
    y_predicted = log_reg.y_pred(x_test_scaled,threshold=0.5)
    spec , G = log_reg.GetScores(y_test,y_predicted)
#log_reg.confusion(y_test,y_predicted,'Default Confusion Matrix')

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
F:\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

Mean accuracy score is 0.830277
Mean accuracy score is 0.832159
Mean accuracy score is 0.833247
Mean accuracy score is 0.834836
Mean accuracy score is 0.832107
Mean accuracy score is 0.834629
Mean accuracy score is 0.835354
Mean accuracy score is 0.830294
Mean accuracy score is 0.832850
Mean accuracy score is 0.831572


C=20

In [19]:
for i in range(1,11):
    not_bi = log_reg.not_bi(x)

    x_train_temp, x_test_real, y_train_temp, y_test_real = log_reg.split(x,y,rand=0)

    x_train, x_test, y_train, y_test = validation_dataset(x_train_temp,y_train_temp,10,i)

    scaler = StandardScaler()
    scaler.fit(x_train[not_bi]) 

    x_train_scaled=x_train
    x_test_scaled=x_test

    x_train_scaled[not_bi] = scaler.transform(x_train[not_bi])
    x_test_scaled[not_bi]  = scaler.transform(x_test[not_bi])

# Fit model
    model = log_reg.reg(x_train_scaled,y_train,20,100)
# Evaluate model
    log_reg.ModelValuation(x_test_scaled,y_test,model)
    y_predicted = log_reg.y_pred(x_test_scaled,threshold=0.5)
    spec , G = log_reg.GetScores(y_test,y_predicted)
#log_reg.confusion(y_test,y_predicted,'Default Confusion Matrix')

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
F:\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

Mean accuracy score is 0.830259
Mean accuracy score is 0.832159
Mean accuracy score is 0.833247
Mean accuracy score is 0.834836
Mean accuracy score is 0.832107
Mean accuracy score is 0.834629
Mean accuracy score is 0.835354
Mean accuracy score is 0.830311
Mean accuracy score is 0.832867
Mean accuracy score is 0.831589


Choose a larger iter

In [13]:
# Normalize
for i in range(1,6):
    not_bi = log_reg.not_bi(x)

    x_train_temp, x_test_real, y_train_temp, y_test_real = log_reg.split(x,y,rand=0)

    x_train, x_test, y_train, y_test = validation_dataset(x_train_temp,y_train_temp,5,i)

    scaler = StandardScaler()
    scaler.fit(x_train[not_bi]) 

    x_train_scaled=x_train
    x_test_scaled=x_test

    x_train_scaled[not_bi] = scaler.transform(x_train[not_bi])
    x_test_scaled[not_bi]  = scaler.transform(x_test[not_bi])

# Fit model
    model = log_reg.reg(x_train_scaled,y_train,1,500)
# Evaluate model
    log_reg.ModelValuation(x_test_scaled,y_test,model)
    y_predicted = log_reg.y_pred(x_test_scaled,threshold=0.5)
    spec , G = log_reg.GetScores(y_test,y_predicted)
#log_reg.confusion(y_test,y_predicted,'Default Confusion Matrix')

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
F:\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-doc

Mean accuracy score is 0.837366
Mean accuracy score is 0.840448
Mean accuracy score is 0.839196
Mean accuracy score is 0.837564
Mean accuracy score is 0.837422


In [28]:
for i in range(1,6):
    not_bi = log_reg.not_bi(x)

    x_train_temp, x_test_real, y_train_temp, y_test_real = log_reg.split(x,y,rand=0)

    x_train, x_test, y_train, y_test = validation_dataset(x_train_temp,y_train_temp,5,i)

    scaler = StandardScaler()
    scaler.fit(x_train[not_bi]) 

    x_train_scaled=x_train
    x_test_scaled=x_test

    x_train_scaled[not_bi] = scaler.transform(x_train[not_bi])
    x_test_scaled[not_bi]  = scaler.transform(x_test[not_bi])

# Fit model
    model = log_reg.reg(x_train_scaled,y_train,1,100)
# Evaluate model
    log_reg.ModelValuation(x_test_scaled,y_test,model)
    y_predicted = log_reg.y_pred(x_test_scaled,threshold=0.5)
    spec , G = log_reg.GetScores(y_test,y_predicted)
#log_reg.confusion(y_test,y_predicted,'Default Confusion Matrix')

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
F:\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

Mean accuracy score is 0.831235
Mean accuracy score is 0.836399
Mean accuracy score is 0.833653
Mean accuracy score is 0.832902
Mean accuracy score is 0.832211


In [29]:
for i in range(1,6):
    not_bi = log_reg.not_bi(x)

    x_train_temp, x_test_real, y_train_temp, y_test_real = log_reg.split(x,y,rand=0)

    x_train, x_test, y_train, y_test = validation_dataset(x_train_temp,y_train_temp,5,i)

    scaler = StandardScaler()
    scaler.fit(x_train[not_bi]) 

    x_train_scaled=x_train
    x_test_scaled=x_test

    x_train_scaled[not_bi] = scaler.transform(x_train[not_bi])
    x_test_scaled[not_bi]  = scaler.transform(x_test[not_bi])

# Fit model
    model = log_reg.reg(x_train_scaled,y_train,1,200)
# Evaluate model
    log_reg.ModelValuation(x_test_scaled,y_test,model)
    y_predicted = log_reg.y_pred(x_test_scaled,threshold=0.5)
    spec , G = log_reg.GetScores(y_test,y_predicted)
#log_reg.confusion(y_test,y_predicted,'Default Confusion Matrix')

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
F:\Anaconda3\lib\site-packages\pandas\core\indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

Mean accuracy score is 0.833705
Mean accuracy score is 0.838013
Mean accuracy score is 0.835924
Mean accuracy score is 0.835147
Mean accuracy score is 0.834715
